In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\sachi\OneDrive\Product Demand Forecasting\DATA\product_info.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ProductID        500 non-null    object 
 1   Category         500 non-null    object 
 2   Brand            500 non-null    object 
 3   Material         500 non-null    object 
 4   Weight(kg)       500 non-null    float64
 5   Warranty(years)  500 non-null    int64  
 6   ProductRating    470 non-null    float64
 7   LaunchYear       480 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 31.4+ KB


In [4]:
df.sample(10)

,ProductID,Category,Brand,Material,Weight(kg),Warranty(years),ProductRating,LaunchYear
133,P0134,Toys,BrandA,Cotton,1.00,3,4.2,2023.0
37,P0038,Toys,BrandD,Glass,4.19,2,2.9,2020.0
499,P0500,Clothing,BrandB,Wood,4.01,2,4.8,2020.0
264,P0265,Beauty,BrandA,Cotton,3.05,0,4.5,2020.0
76,P0077,Clothing,BrandA,Wood,2.76,3,3.6,2021.0
175,P0176,Electronics,BrandB,Wood,3.68,1,4.3,2021.0
46,P0047,Electronics,BrandC,Plastic,1.24,2,3.9,2018.0
144,P0145,Beauty,BrandB,Glass,3.05,1,4.1,2021.0
376,P0377,Beauty,BrandB,Metal,2.70,2,2.9,2022.0
374,P0375,Clothing,BrandC,Metal,2.98,0,4.6,2023.0


In [5]:
df.isnull().sum()

ProductID           0
Category            0
Brand               0
Material            0
Weight(kg)          0
Warranty(years)     0
ProductRating      30
LaunchYear         20
dtype: int64

In [6]:
df.duplicated(subset=["ProductID"]).sum()

np.int64(0)

In [7]:
print(df[df['ProductRating'].isnull()])



    ProductID     Category   Brand Material  Weight(kg)  Warranty(years)  \
9       P0010         Toys  BrandD   Cotton        2.71                3   
14      P0015       Beauty  BrandD  Plastic        4.29                3   
33      P0034     Clothing  BrandA   Cotton        0.51                2   
54      P0055         Home  BrandB    Metal        4.51                0   
62      P0063         Toys  BrandA   Cotton        2.08                1   
124     P0125     Clothing  BrandB     Wood        2.27                1   
131     P0132         Toys  BrandA  Plastic        3.90                3   
146     P0147       Beauty  BrandC     Wood        0.52                1   
163     P0164     Clothing  BrandB    Metal        0.83                3   
164     P0165         Home  BrandB     Wood        4.37                1   
168     P0169     Clothing  BrandA  Plastic        4.57                3   
189     P0190         Home  BrandD    Glass        0.29                1   
227     P022

# Filling Missing Values

In [8]:
# filling by Category, Brand, Material 
# df['ProductRating'] = df.groupby(['Category', 'Brand'])['ProductRating'].transform(
#     lambda x: x.fillna(x.mean())
# )
df['ProductRating'] = df.groupby(['Category', 'Brand', 'Material'])['ProductRating'].transform(
    lambda x: x.fillna(x.mean())
)


In [9]:
df.isnull().sum()

ProductID           0
Category            0
Brand               0
Material            0
Weight(kg)          0
Warranty(years)     0
ProductRating       0
LaunchYear         20
dtype: int64

In [10]:
df[(df['Category'] == 'Toys') & (df['Brand'] == 'BrandD') & (df['Material']=='Cotton')]['ProductRating']


9      4.1
178    3.9
301    3.9
308    4.5
Name: ProductRating, dtype: float64

In [11]:
df[(df['Category'] == 'Toys') & (df['Brand'] == 'BrandD') & (df['Material']=='Cotton')]['ProductRating'].mean()

np.float64(4.1)

In [12]:
# Fill missing LaunchYear using mode of Brand + Category group
df['LaunchYear'] = df.groupby(['Brand', 'Category'])['LaunchYear'].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)


In [13]:
df.isnull().sum()

ProductID          0
Category           0
Brand              0
Material           0
Weight(kg)         0
Warranty(years)    0
ProductRating      0
LaunchYear         0
dtype: int64

In [14]:
df.describe()

,Weight(kg),Warranty(years),ProductRating,LaunchYear
count,500.000000,500.000000,500.000000,500.000000
mean,2.624920,1.478000,3.802753,2020.502000
std,1.391958,1.145487,0.718118,1.745879
min,0.200000,0.000000,2.500000,2018.000000
25%,1.405000,0.000000,3.200000,2019.000000
50%,2.710000,1.000000,3.800000,2020.000000
75%,3.822500,3.000000,4.400000,2022.000000
max,4.980000,3.000000,5.000000,2023.000000


In [15]:
df.describe(include='object')

,ProductID,Category,Brand,Material
count,500,500,500,500
unique,500,5,4,5
top,P0500,Beauty,BrandA,Metal
freq,1,112,143,108


In [16]:
df.nunique()

ProductID          500
Category             5
Brand                4
Material             5
Weight(kg)         309
Warranty(years)      4
ProductRating       44
LaunchYear           6
dtype: int64

In [17]:
# Checking fr Negative Values
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    neg_rows = df[df[col] < 0]
    if not neg_rows.empty:
        print(f"Negative values found in '{col}':")
        print(neg_rows)
    else:
        print("No Negative Values Found")    


No Negative Values Found
No Negative Values Found
No Negative Values Found
No Negative Values Found


In [18]:
# Invalid Warranty
count=df[df['Warranty(years)']>5].shape[0]
if count>0:
    print("There are Invalid Warranty")
else:
    print(" There is no Invalid Entry")    

 There is no Invalid Entry


# Outlier Detection

In [19]:
df['Weight(kg)'].describe()


count    500.000000
mean       2.624920
std        1.391958
min        0.200000
25%        1.405000
50%        2.710000
75%        3.822500
max        4.980000
Name: Weight(kg), dtype: float64

In [20]:
Q1 = df['Weight(kg)'].quantile(0.25)
Q3 = df['Weight(kg)'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['Weight(kg)'] < Q1 - 1.5*IQR) | (df['Weight(kg)'] > Q3 + 1.5*IQR)]
print(outliers)


Empty DataFrame
Columns: [ProductID, Category, Brand, Material, Weight(kg), Warranty(years), ProductRating, LaunchYear]
Index: []


In [21]:
# Convert to 'category' dtype to save memory
df['Category'] = df['Category'].astype('category')
df['Brand'] = df['Brand'].astype('category')
df['Material'] = df['Material'].astype('category')
df['Warranty(years)'] = df['Warranty(years)'].astype('category')  # optional

# Convert LaunchYear to integer (if no NaN left)
df['LaunchYear'] = df['LaunchYear'].astype(int)


In [22]:
df.dtypes

ProductID            object
Category           category
Brand              category
Material           category
Weight(kg)          float64
Warranty(years)    category
ProductRating       float64
LaunchYear            int64
dtype: object

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   ProductID        500 non-null    object  
 1   Category         500 non-null    category
 2   Brand            500 non-null    category
 3   Material         500 non-null    category
 4   Weight(kg)       500 non-null    float64 
 5   Warranty(years)  500 non-null    category
 6   ProductRating    500 non-null    float64 
 7   LaunchYear       500 non-null    int64   
dtypes: category(4), float64(2), int64(1), object(1)
memory usage: 18.5+ KB


In [24]:
df.to_csv(r'C:\Users\sachi\OneDrive\Product Demand Forecasting\CLEANED DATA\product_info_cleaned.csv', index=False)